In [8]:

from importlib import reload
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import numpy as np
import sys


# Visualizations will be shown in the notebook:

%matplotlib inline


# Import local deps

import src.helpers.lineRegressorWithMemory as LRM
import src.helpers.imageProcessing as IP
import src.helpers.plot as P


if "src.helpers.imageProcessing" in sys.modules:
    print("Reloading Modules...")
    
    reload(LRM)
    reload(IP)
    reload(P)
    
    print("\nModules Reloaded!")


Reloading Modules...

Modules Reloaded!


In [9]:

# IMAGE PROCESSING:

rightLineStream = LRM.LineStream()
leftLineStream = LRM.LineStream()

def process_image(imageRGB):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    imageHLS = IP.rgb2hls(imageRGB)
    imageYW = IP.yellowAndWhiteHlsFilter(imageHLS)
    
    WIDTH = imageYW.shape[1]
    HEIGHT = imageYW.shape[0]
    
    REGION_MASK = np.array([[
        # OUTER SHAPE:
        
        (0.05 * WIDTH, 0.90 * HEIGHT),
        (0.45 * WIDTH, 0.60 * HEIGHT),
        (0.55 * WIDTH, 0.60 * HEIGHT),
        (0.95 * WIDTH, 0.90 * HEIGHT),
        
        # INNER CUTOUT:
        
        (0.70 * WIDTH, 0.90 * HEIGHT),
        (0.50 * WIDTH, 0.70 * HEIGHT),
        (0.30 * WIDTH, 0.90 * HEIGHT),
    ]], dtype=np.int32)
    
    imageRegion = IP.regionOfInterestFilter(imageYW, REGION_MASK)
    imageBlur = IP.gaussianBlur(imageRegion, 5)
    imageCanny = IP.canny(imageBlur, 50, 100)
    imagesHoughLines = IP.houghLines(imageCanny, 2, np.pi / 180, 30, 20, 80)
        
    imageLines = np.zeros((imageRGB.shape[0], imageRGB.shape[1], 3), dtype=np.uint8)
    
    if imagesHoughLines is None:
        rightLineStream.drawLineFromPoints(imageLines, [[]], [], [])
        leftLineStream.drawLineFromPoints(imageLines, [[]], [], [])
    else:
        linesPoints = IP.classifyLinesPoints(imagesHoughLines, WIDTH)
        right = linesPoints['right']
        left = linesPoints['left']
        
        rightLineStream.drawLineFromPoints(imageLines, right['X'], right['Y'], right['weights'])
        leftLineStream.drawLineFromPoints(imageLines, left['X'], left['Y'], left['weights'])
        
    return IP.weighted_img(imageLines, imageRGB)


# VIDEO PROCESSING:

outputDir = "./output/videos/"
videosDir = "./input/videos/"
videosNames = os.listdir(videosDir)

def process_video(videoName):
    print("--------\n\nProcessing %s:" % videoName)

    rightLineStream.reset()
    leftLineStream.reset()
    
    inputClip = VideoFileClip(videosDir + videoName)
    outputClip = inputClip.fl_image(process_image) # NOTE: this function expects color images!
    %time outputClip.write_videofile(outputDir + videoName, audio=False)
    
    return outputClip
    
print("Available videos:\n")

for videoName in videosNames: print(videoName)
    

Available videos:

001 - Solid White Right.mp4
002 - Solid Yellow Left.mp4
003 - Challenge.mp4
004 - Advanced Project.mp4
005 - Advanced Project Challenge.mp4
006 - Advanced Project Harder Challenge.mp4


In [10]:

for videoName in videosNames:
    output = process_video(videoName)


--------

Processing 001 - Solid White Right.mp4:


[MoviePy] >>>> Building video ./output/videos/001 - Solid White Right.mp4


[MoviePy] Writing video ./output/videos/001 - Solid White Right.mp4


  0%|          | 0/222 [00:00<?, ?it/s]

  4%|▍         | 9/222 [00:00<00:02, 89.16it/s]

...

100%|█████████▉| 1198/1200 [00:41<00:00, 31.55it/s]

100%|█████████▉| 1199/1200 [00:41<00:00, 29.10it/s]

[MoviePy] Done.


[MoviePy] >>>> Video ready: ./output/videos/006 - Advanced Project Harder Challenge.mp4 



CPU times: user 40.3 s, sys: 8.83 s, total: 49.1 s
Wall time: 42.2 s
